In [1]:
#Cargamos las librerías
import numpy as np
import bokeh.plotting as bkp
import bokeh.models as bkm
import bokeh.io as bkio
import bokeh.layouts as bkl
from bokeh.colors import RGB
import pandas as pd
import datetime as dt

#Mostramos las gráficas dentro del notebook
bkp.output_notebook()

Loading BokehJS ...

In [2]:
#Cargamos los datos

consumo=pd.read_excel("1202002.xlsx", converters={'0':'.'}, skiprows=6, skipfooter=7,index_col=None)
carbonh = consumo[(consumo[consumo.columns[0]]=="Consumo interior bruto de: carbón")]
carbonm = pd.melt(carbonh, id_vars=[carbonh.columns[0]], var_name = "Fecha", value_name="Tep")
petroleoh = consumo[(consumo[consumo.columns[0]]=="Consumo interior bruto de: petróleo")]
petroleom = pd.melt(petroleoh, id_vars=[carbonh.columns[0]], var_name = "Fecha", value_name="Tep")
gash = consumo[(consumo[consumo.columns[0]]=="Consumo interior bruto de: gas natural")]
gasm = pd.melt(gash, id_vars=[carbonh.columns[0]], var_name = "Fecha", value_name="Tep")
hidroh = consumo[(consumo[consumo.columns[0]]=="Consumo(=Producción) interior: hidráulica")]
hidrom = pd.melt(hidroh, id_vars=[carbonh.columns[0]], var_name = "Fecha", value_name="Tep")
nuclearh = consumo[(consumo[consumo.columns[0]]=="Consumo(=Producción) interior: nuclear")]
nuclearm = pd.melt(nuclearh, id_vars=[nuclearh.columns[0]], var_name = "Fecha", value_name="Tep")
solareolicah = consumo[(consumo[consumo.columns[0]]=="Consumo(=Producción) interior: eólica y solar")]
solareolicam = pd.melt(nuclearh, id_vars=[solareolicah.columns[0]], var_name = "Fecha", value_name="Tep")
biomasah = consumo[(consumo[consumo.columns[0]]=="Consumo interior: biomasa y residuos")]
biomasam = pd.melt(biomasah, id_vars=[nuclearh.columns[0]], var_name = "Fecha", value_name="Tep")
biomasam["Tep"].loc[(biomasam["Tep"]=='.')]=0

In [3]:
carbonm['Fecha2']=pd.to_datetime(carbonm['Fecha'],format="%YM%m")
petroleom['Fecha2']=pd.to_datetime(petroleom['Fecha'],format="%YM%m")
gasm['Fecha2']=pd.to_datetime(gasm['Fecha'],format="%YM%m")
hidrom['Fecha2']=pd.to_datetime(hidrom['Fecha'],format="%YM%m")
nuclearm['Fecha2']=pd.to_datetime(nuclearm['Fecha'],format="%YM%m")
solareolicam['Fecha2']=pd.to_datetime(solareolicam['Fecha'],format="%YM%m")
biomasam['Fecha2']=pd.to_datetime(biomasam['Fecha'],format="%YM%m")

carbon = carbonm.resample('AS', on='Fecha2')['Tep'].agg(['sum'])
petroleo=petroleom.resample('AS', on='Fecha2')['Tep'].agg(['sum'])
gas=gasm.resample('AS', on='Fecha2')['Tep'].agg(['sum'])
hidro=hidrom.resample('AS', on='Fecha2')['Tep'].agg(['sum'])
nuclear=nuclearm.resample('AS', on='Fecha2')['Tep'].agg(['sum'])
solareolica=solareolicam.resample('AS', on='Fecha2')['Tep'].agg(['sum'])
biomasa=biomasam.resample('AS', on='Fecha2')['Tep'].agg(['sum'])

nuclear['Fecha3']=nuclear


In [4]:
nuclear.index

DatetimeIndex(['1995-01-01', '1996-01-01', '1997-01-01', '1998-01-01',
               '1999-01-01', '2000-01-01', '2001-01-01', '2002-01-01',
               '2003-01-01', '2004-01-01', '2005-01-01', '2006-01-01',
               '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01',
               '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01',
               '2015-01-01'],
              dtype='datetime64[ns]', name='Fecha2', freq='AS-JAN')

In [5]:
#Cargamos los colores elegidos como paleta para este MOOC
p_azulmarino = RGB(0, 28, 100)
p_azulclaro = RGB(0, 131, 255)
p_amarillo = RGB(244, 255, 84)
p_verde = RGB(172, 255, 165)
p_rojo = RGB(255, 67, 67)

In [6]:
#Creamos una figura de Bokeh que va a contener nuestra gráfica. Indico el tamaño, que el eje X van a ser fechas
#y las herramientas activas (mover la gráfica y hacer zoom con el ratón)

p = bkp.figure(tools=["pan","wheel_zoom","box_zoom","save,reset"], width=900, height=500,x_axis_type="datetime",
               active_drag = "pan", active_scroll = "wheel_zoom")

año = carbon.index

banda_superior=carbon["sum"]
banda_inferior=np.zeros(len(carbon))
carbon_x = np.append(año, año[::-1])
carbon_y = np.append(banda_inferior, banda_superior[::-1])

p.patch(carbon_x, carbon_y, color=p_azulmarino, fill_alpha=0.6, line_alpha=0.3, legend="Carbón")

banda_inferior = banda_superior
banda_superior = banda_superior + petroleo["sum"]
petroleo_x = np.append(año, año[::-1])
petroleo_y = np.append(banda_inferior, banda_superior[::-1])

p.patch(petroleo_x, petroleo_y, color=p_azulmarino, fill_alpha=1, line_alpha=0.3, legend="Petróleo")

banda_inferior = banda_superior
banda_superior = banda_superior + gas["sum"]
gas_x = np.append(año, año[::-1])
gas_y = np.append(banda_inferior, banda_superior[::-1])

p.patch(gas_x, gas_y, color=p_amarillo, fill_alpha=1, line_alpha=0.3, legend="Gas Natural")

banda_inferior = banda_superior
banda_superior = banda_superior + nuclear["sum"]
nuclear_x = np.append(año, año[::-1])
nuclear_y = np.append(banda_inferior, banda_superior[::-1])

p.patch(nuclear_x, nuclear_y, color=p_rojo, fill_alpha=1, line_alpha=0.3, legend="Nuclear")

banda_inferior = banda_superior
banda_superior = banda_superior + hidro["sum"]
hidro_x = np.append(año, año[::-1])
hidro_y = np.append(banda_inferior, banda_superior[::-1])

p.patch(hidro_x, hidro_y, color=p_azulclaro, fill_alpha=1, line_alpha=0.3, legend="Hidroeléctrica")

banda_inferior = banda_superior
banda_superior = banda_superior + solareolica["sum"]
solareolica_x = np.append(año, año[::-1])
solareolica_y = np.append(banda_inferior, banda_superior[::-1])

p.patch(solareolica_x, solareolica_y, color=p_verde, fill_alpha=1, line_alpha=0.3, legend="Solar y Eólica")

banda_inferior = banda_superior
banda_superior = banda_superior + biomasa["sum"]
biomasa_x = np.append(año, año[::-1])
biomasa_y = np.append(banda_inferior, banda_superior[::-1])

p.patch(biomasa_x, biomasa_y, color=p_rojo, fill_alpha=0.5, line_alpha=0.3, legend="Biomasa")

GlyphRenderer(id='5c75585f-815f-4db5-b35a-c6f973853999', ...)

In [7]:
#Indico las propiedades del gráfico
p.title.text = "Consumo Energético en España"
p.title.text_color = "black"
p.title.text_font = "ADAM CG PRO"
p.title.text_font_style = "normal"
p.title.text_font_size = "16pt"
p.title.align = "center"

p.background_fill_color = "white"
p.background_fill_alpha = 1.

p.xaxis.axis_label = 'Año'
p.xaxis.axis_label_text_font = "ADAM CG PRO"
p.xaxis.axis_label_text_font_style  = "normal"
p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label = 'Tonelada Equivalentes de Petróleo'
p.yaxis.axis_label_text_font = "ADAM CG PRO"
p.yaxis.axis_label_text_font_style  = "normal"
p.yaxis.axis_label_text_font_size = "12pt"

p.yaxis[0].formatter = bkm.NumeralTickFormatter(format="0,0")

p.y_range = bkm.Range1d(start=0, end=200000)

p.ygrid.grid_line_color=p_azulclaro
p.ygrid.grid_line_alpha=0.25
p.xgrid.grid_line_color=p_azulclaro
p.xgrid.grid_line_alpha=0.0

p.xaxis.major_tick_line_color = p_rojo
p.xaxis.major_tick_line_width = 3
p.xaxis.minor_tick_line_color = p_rojo
p.xaxis.minor_tick_line_width = 1
p.yaxis.major_tick_line_color = p_rojo
p.yaxis.major_tick_line_width = 3
p.yaxis.minor_tick_line_color = p_rojo
p.yaxis.minor_tick_line_width = 1

p.outline_line_width = 7
p.outline_line_alpha = 0.5
p.outline_line_color = p_azulclaro

p.legend.location = "top_left"
#p.legend.click_policy="hide"
p.legend.label_text_font = "ADAM CG PRO"
p.legend.orientation = "horizontal"

In [8]:
#Si deseamos guardar la gráfica como un fichero html, descomentar la siguiente línea de código,
#al ejecutarse el notebook generará un html en el directorio
#p.output_backend = "canvas"
#bkp.output_file("consumoespana.html", title='Bokeh Plot', mode='cdn', root_dir=None)

#para guardar un svg usando el menú de la gráfica (en lugar de un PNG), descomentar esta línea
#p.output_backend = "svg"

bkp.show(p)